In [14]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [20]:
trX = trX.reshape(-1, 28, 28, 1) #28*28*1 input img
teX = teX.reshape(-1, 28, 28, 1) #28*28*1 input img

X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

In [24]:
#定义初始化权重参数
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev = 0.01))
w = init_weights([3, 3, 1, 32])     #patch大小为3*3, 输入维度为1, 输出维度为32
w2 = init_weights([3, 3, 32, 64])    #patch大小为3*3, 输入维度为32, 输出维度64
w3 = init_weights([3, 3, 64, 128])   #patch大小为3*3, 输入维度为64, 输出维度为128
w4 = init_weights([128*4*4, 625])    # 全连接层,输入维度为 128 × 4 × 4, 是上一层的输
                                    #出数据又三维的转变成一维, 输出维度为 625
w_o = init_weights([625, 10]) #输出层,输入维度为625， 输出维度为10，代表10类标签

In [29]:
#定义神经网络模型参数
# X:输入数据
# w: 每一层的权重
# p_keep_conv, p_keep_hidden: dropout 要保留的神经元比例

def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_kidden):
    #第一组卷积层及池化层， 最后dropout 一些神经元
    l1a = tf.nn.relu(tf.nn.conv2d(X, w, strides = [1, 1, 1, 1], padding = 'SAME'))
    # lla shape = (?, 28, 28, 32)
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME')
    # l1 shape = (?, 14, 14, 32)
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    #第二组卷积层及池化层，最后dropout一些神经元
    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides = [1, 1, 1, 1], padding = 'SAME'))
    #l2a shape(?, 14, 14, 64)
    l2 = tf.nn.max_pool(l2a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    # l2 shape=(?, 7, 7, 64)
    l2 = tf.nn.dropout(l2, p_keep_conv)
    
    #第三租卷积层及池化层，最后dropout一些神经元
    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides = [1, 1, 1, 1], padding = 'SAME'))
    #l3a shape=(?, 7, 7, 128)
    l3 = tf.nn.max_pool(l3a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    # l3 shape = (?, 4, 4, 128)
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]]) # reshape to (？, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    #全连接层，最后dropout一些神经元
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)
    # l4 shape = (?,625)
    #输出层
    pyx = tf.matmul(l4, w_o)
    # pyx shape = (?,10)
    return pyx #返回预测值
    

In [31]:
#确定预测值
p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden) #得到预测值
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = py_x, labels = Y))
train_op = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [35]:
#定义模型和评估模型
batch_size = 128
test_size = 256

#启动会话图，开始训练和评估:
#Launch the graph in a session
with tf.Session() as sess:
    # you need to initializer all variables
    tf.global_variables_initializer().run()
    for i in range(100):
        training_batch = zip(range(0, len(trX), batch_size),
                            range(batch_size, len(trX) + 1, batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict = {X: trX[start:end], Y:trY[start:end],
                                           p_keep_conv: 0.8, p_keep_hidden: 0.5})
        test_indices = np.arange(len(teX)) #Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0: test_size]
        print(i, np.mean(np.argmax(teY[test_indices], axis = 1) == 
                        sess.run(predict_op, feed_dict = { X: teX[test_indices],
                                                                p_keep_conv: 1.0,
                                                                p_keep_hidden: 1.0})))

(0, 0.94140625)
(1, 0.97265625)
(2, 0.9765625)
(3, 0.98828125)
(4, 0.9921875)
(5, 1.0)
(6, 0.9921875)
(7, 0.9921875)
(8, 0.99609375)


KeyboardInterrupt: 